In [17]:
import ftplib
import os
import pandas as pd
import math

In [18]:
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [20]:
for x in range(1904, 2020):
    
    airData = airData.drop(airData.index, inplace=True)
    soilData = soilData.drop(soilData.index, inplace=True)
    
    midasEnd = str(x) + "12"
    midasStart = str(x) + "01"
    
    #get data
    airData = pd.read_csv("airCSVData/midas_tempdrnl_"+midasStart+"-"+midasEnd+".csv",sep=r'\s*,\s*',
                           header=0, encoding='ascii', engine='python')
    soilData = pd.read_csv("soilCSVData/midas_soiltemp_"+midasStart+"-"+midasEnd+".csv",sep=r'\s*,\s*',
                           header=0, encoding='ascii', engine='python')
    
    #print("got midas "+midasStart+"-"+midasEnd+".csv")
    
    #remove unnedded columns
    airDataS = airData.drop(columns = ['MAX_AIR_TEMP_J','MIN_AIR_TEMP_J','MIN_GRSS_TEMP_J','MIN_CONC_TEMP_J',
                                 'MAX_AIR_TEMP_Q','MIN_AIR_TEMP_Q','MIN_GRSS_TEMP_Q','MIN_CONC_TEMP_Q',
                                 'ID_TYPE','VERSION_NUM','MAX_AIR_TEMP','MIN_AIR_TEMP',
                                 'MET_DOMAIN_NAME','ID','MIDAS_STMP_ETIME','MIN_CONC_TEMP','REC_ST_IND',
                                 'OB_HOUR_COUNT','METO_STMP_TIME'])
    soilData = soilData.drop(columns = ['Q100CM_SOIL_TEMP_J','Q50CM_SOIL_TEMP_J','Q30CM_SOIL_TEMP_J','Q20CM_SOIL_TEMP_J',
                        'Q10CM_SOIL_TEMP_J','Q100CM_SOIL_TEMP_Q','Q50CM_SOIL_TEMP_Q','Q30CM_SOIL_TEMP_Q',
                        'Q20CM_SOIL_TEMP_Q','Q10CM_SOIL_TEMP_Q','Q5CM_SOIL_TEMP_Q','Q5CM_SOIL_TEMP_J',
                        'Q50CM_SOIL_TEMP','Q30CM_SOIL_TEMP','Q20CM_SOIL_TEMP',
                        'Q10CM_SOIL_TEMP','Q5CM_SOIL_TEMP', 'ID', 'ID_TYPE', 'MET_DOMAIN_NAME', 
                        'VERSION_NUM', 'MIDAS_STMP_ETIME','METO_STMP_TIME','REC_ST_IND'])
    
    #split date/time column
    airDataS[['OB_END_DATE','OB_END_TIME']] = airDataS['OB_END_TIME'].str.split(' ',expand=True)
    soilData[['OB_DATE','OB_TIME']] = soilData['OB_TIME'].str.split(' ',expand=True)
    
    #create list of stations with valid data
    
    
    cmTest = (soilData['Q100CM_SOIL_TEMP'] > 0)
    
    filteredAirSet = airDataS[airDataS.MIN_GRSS_TEMP.notnull()]
    filteredSet = soilData[cmTest]
    
    noOfStationsGrass = filteredAirSet.SRC_ID.unique().tolist()
    noOfStationsSoil = filteredSet.SRC_ID.unique().tolist()

    bothHaveData = list(set(noOfStationsSoil) & set(noOfStationsGrass))
    
    stationID = {}

    for stationDF in bothHaveData:
        stationID[stationDF] = pd.DataFrame(columns = ['OB_TIME','SRC_ID','Q100CM_SOIL_TEMP','OB_DATE','MIN_GRSS_TEMP'])

    counter = 0;
    start_date = date(x, 1, 1)
    end_date = date(x, 12,31)
    #12, 31)
    for single_date in daterange(start_date, end_date):
        getDaySoil = filteredSet['OB_DATE'] == single_date.strftime("%Y-%m-%d")
        getDayAir = filteredAirSet['OB_END_DATE'] == single_date.strftime("%Y-%m-%d")

        for stationNo in bothHaveData:

            getStationS = filteredSet['SRC_ID'] == stationNo
            getStationA = filteredAirSet['SRC_ID'] == stationNo

            stationFilterS = getDaySoil & getStationS
            #print(filteredSet[stationFilterS])

            stationFilterA = getDayAir & getStationA
            #print(filteredAirSet[stationFilterA])
            oneValSoil = filteredSet[stationFilterS]
            oneValAir = filteredAirSet[stationFilterA]

            try:
                listForAppend = [oneValSoil['OB_TIME'].values[0],
                                        oneValSoil['SRC_ID'].values[0],
                                        oneValSoil['Q100CM_SOIL_TEMP'].values[0],
                                        oneValSoil['OB_DATE'].values[0],
                                        oneValAir['MIN_GRSS_TEMP'].values[0]]

                s = pd.Series(listForAppend, index=stationID[stationNo].columns)

                stationID[stationNo] = stationID[stationNo].append(s, ignore_index=True)
            except:
                #print("no val " + str(stationNo))
                counter += 1
                
    for stationDF in bothHaveData:
        stationID[stationDF].to_csv("stationData/stationID_"+str(math.trunc(stationDF))+".csv", mode='a', header=False)

    print("Written station data for year " + str(x))


Written station data for year 1904
Written station data for year 1905
Written station data for year 1906
Written station data for year 1907
Written station data for year 1908
Written station data for year 1909
Written station data for year 1910
Written station data for year 1911
Written station data for year 1912
Written station data for year 1913
Written station data for year 1914
Written station data for year 1915


KeyboardInterrupt: 